# Importing Library 

In [1]:
import pandas as pd
import numpy as np
import math
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Import dataset

In [2]:
# Import dataset
apple_df= pd.read_csv("./stock_market_data/sp500/csv/AAPL.csv")
apple_df.head()

,Date,Low,Open,Volume,High,Close,Adjusted Close
0,12-12-1980,0.128348,0.128348,469033600,0.128906,0.128348,0.099874
1,15-12-1980,0.121652,0.122210,175884800,0.122210,0.121652,0.094663
2,16-12-1980,0.112723,0.113281,105728000,0.113281,0.112723,0.087715
3,17-12-1980,0.115513,0.115513,86441600,0.116071,0.115513,0.089886
4,18-12-1980,0.118862,0.118862,73449600,0.119420,0.118862,0.092492


# Convert date from string to date format

In [3]:
# converting date format to datetime
apple_df['Date'] = pd.to_datetime(apple_df.Date)
apple_df.head()

/var/folders/vq/cp7zz90136g6n1v4853v79qr0000gn/T/ipykernel_35112/213034236.py:2: UserWarning: Parsing '15-12-1980' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  apple_df['Date'] = pd.to_datetime(apple_df.Date)
/var/folders/vq/cp7zz90136g6n1v4853v79qr0000gn/T/ipykernel_35112/213034236.py:2: UserWarning: Parsing '16-12-1980' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  apple_df['Date'] = pd.to_datetime(apple_df.Date)
/var/folders/vq/cp7zz90136g6n1v4853v79qr0000gn/T/ipykernel_35112/213034236.py:2: UserWarning: Parsing '17-12-1980' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  apple_df['Date'] = pd.to_datetime(apple_df.Date)
/var/folders/vq/cp7zz90136g6n1v4853v79qr0000gn/T/ipykernel_35112/213034236.py:2: UserWarning: Parsing '18-12-1980' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent 

,Date,Low,Open,Volume,High,Close,Adjusted Close
0,1980-12-12,0.128348,0.128348,469033600,0.128906,0.128348,0.099874
1,1980-12-15,0.121652,0.122210,175884800,0.122210,0.121652,0.094663
2,1980-12-16,0.112723,0.113281,105728000,0.113281,0.112723,0.087715
3,1980-12-17,0.115513,0.115513,86441600,0.116071,0.115513,0.089886
4,1980-12-18,0.118862,0.118862,73449600,0.119420,0.118862,0.092492


# Sorting dataset by date format

In [4]:
# sorting by date 
apple_df.sort_values(by='Date', inplace=True)
apple_df.head()
apple_df = apple_df[(apple_df['Date'] > "2019-09-09") & (apple_df['Date'] < "2021-09-09")] 
apple_df

,Date,Low,Open,Volume,High,Close,Adjusted Close
9789,2019-09-10,56.410000,56.757500,74770400,56.947498,56.757500,55.437607
9831,2019-09-12,66.227501,67.500000,128042400,67.699997,66.730003,65.373894
9771,2019-09-13,54.255001,55.000000,159053200,55.197498,54.687500,53.415745
9772,2019-09-16,54.389999,54.432499,84632400,55.032501,54.974998,53.696564
9773,2019-09-17,54.779999,54.990002,73274800,55.205002,55.174999,53.891907
...,...,...,...,...,...,...,...
10144,2021-09-03,118.790001,119.029999,129525800,122.059998,121.089996,119.832428
10166,2021-09-04,129.470001,129.800003,106686700,133.039993,133.000000,131.618729
10208,2021-09-06,126.519997,127.209999,56877900,127.750000,127.129997,126.023407
10229,2021-09-07,142.649994,142.750000,99890800,145.649994,145.110001,143.846893


In [5]:
apple_df.shape

(504, 7)

In [6]:
# making a separate dataframe consisting of only closing prices
closedf = apple_df[['Date','Close']]
print("Shape of close dataframe:", closedf.shape)

Shape of close dataframe: (504, 2)


# Plotting close prices

In [7]:
fig = px.line(closedf, x=closedf.Date, y=closedf.Close,labels={'Date':'Date','Close':'Close Stock'})
fig.update_traces(marker_line_width=2, opacity=0.6)
fig.update_layout(title_text='Stock close price chart', plot_bgcolor='white', font_size=15, font_color='black')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [8]:
close_stock = closedf.copy() # making a copy of dataframe
del closedf['Date'] # removing date column
scaler=MinMaxScaler(feature_range=(0,1)) # scaling values between 0 and 1
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

(504, 1)


# Split data for training and testing

Ratio for training and testing data is 70:30

In [9]:
# splitting dataset into train and test split with 70% data as training
training_size=int(len(closedf)*0.7)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

train_data:  (352, 1)
test_data:  (152, 1)


In [10]:
def create_dataset(dataset, time_step=1):
    """
    creates a dataset according to the time series prediction model
    converts an array of values into a dataset matrix
    """
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]  #i=0, 0,1,2,3-----99,100
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [11]:
# reshaping 
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

X_train:  (336, 15)
y_train:  (336,)
X_test:  (136, 15)
y_test (136,)


In [12]:
# importing random forest regressor
from sklearn.ensemble import RandomForestRegressor
# creating a regressor object 
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0) 
regressor.fit(X_train, y_train) # fitting the training data

RandomForestRegressor(random_state=0)

In [13]:
# predictions
train_predict=regressor.predict(X_train)
test_predict=regressor.predict(X_test)
 # reshaping for plotting
train_predict = train_predict.reshape(-1,1)
test_predict = test_predict.reshape(-1,1)
print("Train data prediction:", train_predict.shape)
print("Test data prediction:", test_predict.shape)

Train data prediction: (336, 1)
Test data prediction: (136, 1)


In [14]:
# transforming into their original forms
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

Evaluation metrices RMSE, MSE and MAE

In [15]:
# calculating performance metrics
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  3.7196494765309995
Train data MSE:  13.835792228257338
Test data MAE:  2.2986088556335083
-------------------------------------------------------------------------------------
Test data RMSE:  10.126427606510314
Test data MSE:  102.5445360698942
Test data MAE:  7.098376330768362


The explained variance score explains the dispersion of errors of a given dataset, scores close to 1.0 are highly desired, indicating better squares of standard deviations of errors.

In [16]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9800303424849922
Test data explained variance regression score: 0.26734656632493636


R-squared (R2) is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model.
1 = Best, 0 or < 0 = worse

In [17]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.9799680612890259
Test data R2 score: 0.2382577458795403


Regression Loss Mean Gamma deviance regression loss (MGD) and Mean Poisson deviance regression loss (MPD)

In [18]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  0.0018044727417129848
Test data MGD:  0.005559445601621723
----------------------------------------------------------------------
Train data MPD:  0.15183401766310348
Test data MPD:  0.7527133828496555


# Comparison between original price vs predicted price from our model

In [19]:
# shifting train predictions for plotting
look_back = time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)
# shifting test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)
names = cycle(['Original close price','Train predicted close price','Test predicted close price'])
# plotting the results 
plotdf = pd.DataFrame({'Date': close_stock['Date'],
                       'original_close': close_stock['Close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['Date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','Date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (504, 1)
Test predicted data:  (504, 1)
